In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [48]:
num_epochs = 200

batch_size = 128

classes = ('plane', 'car' , 'bird','cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [50]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
cifar_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,])

In [52]:
train_dataset = torchvision.datasets.CIFAR10(root= './data', train = True, download=True, transform = cifar_transforms)
test_dataset =  torchvision.datasets.CIFAR10(root= './data', train = False, download=True, transform = cifar_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [53]:
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset = train_dataset, lengths = [train_size, val_size])
len(train_dataset), len(val_dataset), len(test_dataset)

(45000, 5000, 10000)

In [54]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset =train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(dataset= test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset = val_dataset, batch_size=batch_size, shuffle=True)

In [58]:
import math
class VGG(nn.Module):
    def __init__(self, features):
        super(VGG,self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Linear(512,10)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def make_layers(cfg):
    layers = []
    in_channels =3
    for out_channels in cfg:
        if out_channels == 'M':
            layers += [nn.MaxPool2d(kernel_size = 2, stride =2)]
        else:
            conv2d = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding =1)
            layers += [conv2d, nn.ReLU(inplace = True)]
            in_channels = out_channels
    return nn.Sequential(*layers)

cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

def vgg16():
    return VGG(make_layers(cfg))

In [60]:
from torchmetrics import Accuracy
model = vgg16()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9, weight_decay = 5e-4)
accuracy = Accuracy(task='multiclass', num_classes=10)
eps= [0.01, 0.03, 0.06, 0.1, 0.3, 0.5]

In [62]:
from tqdm.notebook import tqdm

# device-agnostic setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
accuracy = accuracy.to(device)
model = model.to(device)

for epoch in tqdm(range(num_epochs)):
    # Training loop
    train_loss, train_acc = 0.0, 0.0
    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)
        X = projected_gradient_descent(model, X, eps[0], eps[0]/10, 40, np.inf )
        model.train()
        
        y_pred = model(X)
        
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        acc = accuracy(y_pred, y)
        train_acc += acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
        
    # Validation loop
    val_loss, val_acc = 0.0, 0.0
    model.eval()
    with torch.inference_mode():
        for X, y in val_dataloader:
            X, y = X.to(device), y.to(device)
            
            y_pred = model(X)
            
            loss = loss_fn(y_pred, y)
            val_loss += loss.item()
            
            acc = accuracy(y_pred, y)
            val_acc += acc
            
        val_loss /= len(val_dataloader)
        val_acc /= len(val_dataloader)
    
    print(f"Epoch: {epoch}| Train loss: {train_loss: .5f}| Train acc: {(100* train_acc): .5f}| Val loss: {val_loss: .5f}| Val acc: {(100*val_acc): .5f}")

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0| Train loss:  2.07583| Train acc:  18.65580| Val loss:  1.87540| Val acc:  23.71094
Epoch: 1| Train loss:  1.81409| Train acc:  27.94720| Val loss:  1.64626| Val acc:  32.40234
Epoch: 2| Train loss:  1.63149| Train acc:  35.83738| Val loss:  1.71327| Val acc:  32.44141
Epoch: 3| Train loss:  1.48174| Train acc:  43.19020| Val loss:  1.48831| Val acc:  42.42188
Epoch: 4| Train loss:  1.28095| Train acc:  53.67149| Val loss:  1.20335| Val acc:  57.46094
Epoch: 5| Train loss:  1.16170| Train acc:  58.56317| Val loss:  1.00691| Val acc:  66.11328
Epoch: 6| Train loss:  1.05645| Train acc:  62.90025| Val loss:  1.02730| Val acc:  65.56641
Epoch: 7| Train loss:  0.99422| Train acc:  65.09011| Val loss:  0.93358| Val acc:  68.76953
Epoch: 8| Train loss:  0.92407| Train acc:  68.20377| Val loss:  0.79054| Val acc:  73.86719
Epoch: 9| Train loss:  0.86868| Train acc:  70.04864| Val loss:  0.88268| Val acc:  73.84766
Epoch: 10| Train loss:  0.81757| Train acc:  71.85305| Val loss:  0.75

In [64]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "vgg_16_cifar_adv.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Saving the model
print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

# Loading the saved model
model_vgg_16_cifar_adv_loaded = vgg16()
model_vgg_16_cifar_adv_loaded.load_state_dict(torch.load(MODEL_SAVE_PATH))

Saving the model: models\vgg_16_cifar_adv.pth


C:\Users\User\AppData\Local\Temp\ipykernel_19616\3212535161.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_vgg_16_cifar_adv_loaded.load_state_dict(torch.load(MOD

<All keys matched successfully>

In [66]:
test_loss, test_acc = 0, 0

model = model_vgg_16_cifar_adv_loaded.to(device)

model.eval()
with torch.inference_mode():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        
        test_loss += loss_fn(y_pred, y)
        test_acc += accuracy(y_pred, y)
        
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

print(f"Test loss: {test_loss: .5f}| Test acc: {(100*test_acc): .5f}")

Test loss:  0.64499| Test acc:  83.28719
